# Utilities

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        super().__init__()
        self.inputs = inputs
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
        
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

def add_one_row(x):
    return torch.cat((x, torch.ones(x.shape[0]).view((x.shape[0],1))), dim=1)

def model_plot_2d(model, add_one=False, N_points=100, x_lim=(-5,5), plot=True): ## Data plot
    x = np.linspace(*x_lim, N_points)
    y = np.linspace(*x_lim, N_points)
    xx, yy = np.meshgrid(x, y)
    if add_one:
        inputs = torch.tensor(np.stack((xx.flatten(), yy.flatten(), np.ones(N_points**2))), dtype=torch.float32).T
    else:
        inputs = torch.tensor(np.stack((xx.flatten(), yy.flatten())), dtype=torch.float32).T
    zz = model(inputs).detach().view((N_points, N_points)).numpy()
    if plot:
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
        surf = ax.plot_surface(xx, yy, zz, linewidth=0, antialiased=False, cmap=cm.coolwarm)
        fig.colorbar(surf)
        plt.show()
    return xx, yy, zz

def model_plot_1d(model, add_one=False, N_points=1000, x_lim=(-10,10)): ## Data plot
    x = np.linspace(*x_lim, N_points)
    if add_one:
        inputs = torch.tensor(np.stack((x, np.ones(N_points))), dtype=torch.float32).T
    else:
        inputs = torch.tensor(x, dtype=torch.float32).view((N_points, -1))
    y = model(inputs).detach().squeeze().numpy()
    plt.plot(x, y)
    plt.show()

def weight_animation_2d(weight_array, name='animation.mp4'):
    fig = plt.figure(figsize=(10, 3))
    ax = fig.add_subplot()
    
    x_min, x_max = np.min(weight_array[:,:,0])-0.2, np.max(weight_array[:,:,0])+0.2
    y_min, y_max = np.min(weight_array[:,:,1])-0.2, np.max(weight_array[:,:,1])+0.2
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_aspect('equal')
    
    points, = ax.plot([], [], 'o', c='blue', ms=3)
    def animate(i):
        points.set_data(weight_array[i,:,0], weight_array[i,:,1])
        return points
    
    time = 20
    fps = 10
    frames = time * fps
    rate = weight_array.shape[0] // frames
    anim = animation.FuncAnimation(fig, lambda i: animate(rate*i), frames, blit=False, interval=1)
    writer = animation.FFMpegWriter(fps=fps)
    anim.save(name, writer = writer)


def circle_to_line(x):
    return 2 * np.arctan( x[:,1] / (1+x[:,0]))

## density of diracs (with position in x on given interval) convolved with gaussians
def gaussian_conv(x, coef=None, scale=1, interval=(-np.pi, np.pi), N_points=1000):
    res = np.zeros(N_points)
    z_min, z_max = interval
    z = np.linspace(z_min, z_max, N_points+1)
    z = 0.5*(z[1:]+z[:-1])
    if coef is None:
        coef = np.ones(len(x)) / len(x)
    for i in range(len(x)):
        res += coef[i] * np.exp(- 0.5 * (z_min + (z-x[i]-z_min) % (z_max-z_min))**2 / scale**2)
    return z, normalize(res, interval=interval)

def Laplacian(f, h=1):
    return (np.roll(f, 1) + np.roll(f, -1) - 2*f) / h**2

def Grad(f, h=1):
    return (f - np.roll(f, 1)) / h

def normalize(f, interval=(-np.pi,np.pi)):
    z_min, z_max = interval
    return len(f) * f / (f.sum() * (z_max-z_min))

def pmax(x, t):
    return torch.maximum(x, t*torch.ones_like(x))

def pmin(x, t):
    return torch.minimum(x, t*torch.ones_like(x))

# Training Routines

In [ ]:
def training_loop(model, train_loader, optimizer, criterion, noisy=False, tau=0.1):
    loss_list = []
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        loss = criterion(inputs, targets, model)
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        if noisy:
            with torch.no_grad():
                for group in optimizer.param_groups:
                    for p in group["params"]:
                        p.add_(tau * torch.randn_like(p))
        if hasattr(model, 'clipper'):
            model.clipper(model)
    return loss_list

class LearningProblem():
    def __init__(self, model, train_loader, optimizer, criterion):
        super().__init__()
        self.model = model
        self.train_loader = train_loader
        self.optimizer = optimizer
        self.criterion = criterion
        self.loss_list = []
        self.state_list = [copy.deepcopy(self.model.state_dict())]

    def train(self, epochs, noisy=False, tau=0.1):
        self.model.train()
        iterator = tqdm(range(epochs))
        for i in iterator:
            loss = training_loop(self.model, self.train_loader, self.optimizer, self.criterion, noisy=noisy, tau=tau)
            self.loss_list.extend(loss)
            self.state_list.append(copy.deepcopy(self.model.state_dict()))
            iterator.set_description(f'log10(loss) = {np.log10(self.loss_list[-1]):.2f}')

# Models

In [ ]:
class ActivationFunction(nn.Module):
    def __init__(self, f):
        super().__init__()
        self.f = f
        
    def forward(self, x):
        return self.f(x)

class VarProModel(nn.Module):
    def __init__(self, feature_model, width, output_dim, VarProTraining=True, clipper=None):
        super().__init__()
        self.feature_model = feature_model
        self.VarProTraining = VarProTraining
        self.width = width
        self.outer = nn.Linear(width, output_dim, bias=False)
        if VarProTraining:
            self.outer.weight.requires_grad = False
        if clipper is not None:
            self.clipper = clipper
            self.clipper(self)

    def forward(self, inputs):
        return self.outer(self.feature_model(inputs)) / self.width ## normalized output function

class SHLFeatureModel(nn.Module):
    def __init__(self, input_dim, width, activation, bias=False):
        super().__init__()
        self.weight = nn.Parameter(data=torch.randn(width, input_dim), requires_grad=True)
        self.activation = activation
        if bias:
            self.bias = nn.Parameter(data=torch.zeros(width), requires_grad=True)
        else:
            self.bias = None

    def forward(self, x):
        return self.activation(nn.functional.linear(x, self.weight, self.bias))

class SignedSHLFeatureModel(nn.Module):
    def __init__(self, input_dim, width, activation, bias=False):
        super().__init__()
        self.weight = nn.Parameter(data=torch.randn(width, input_dim), requires_grad=True)
        self.sign = nn.Parameter(data=torch.randn(width), requires_grad=True)
        self.activation = activation
        if bias:
            self.bias = nn.Parameter(data=torch.zeros(width), requires_grad=True)
        else:
            self.bias = None

    def forward(self, x):
        return self.activation(nn.functional.linear(x, self.weight, self.bias)) @ torch.diag(self.sign)

class ConvolutionFeatureModel(nn.Module):
    def __init__(self, input_dim, width, activation, scale):
        super().__init__()
        self.weight = nn.Parameter(data=torch.randn(width, input_dim), requires_grad=True)
        self.activation = activation
        self.scale = scale

    def forward(self, x):
        return self.activation(torch.linalg.vector_norm(self.weight.T[None,:,:] - x[:,:,None], dim=1) / self.scale)

def SHL(input_dim, width, activation, bias=False, VarProTraining=True, clipper=None):
    feature_model = SHLFeatureModel(input_dim, width, activation, bias=bias)
    return VarProModel(feature_model, width, 1, VarProTraining=VarProTraining, clipper=clipper)

def SignedSHL(input_dim, width, activation, bias=False, VarProTraining=True, clipper=None):
    feature_model = SignedSHLFeatureModel(input_dim, width, activation, bias=bias)
    return VarProModel(feature_model, width, 1, VarProTraining=VarProTraining, clipper=clipper)

def Convolution(input_dim, width, activation, scale, VarProTraining=True, clipper=None):
    feature_model = ConvolutionFeatureModel(input_dim, width, activation, scale)
    return VarProModel(feature_model, width, 1, VarProTraining=VarProTraining, clipper=clipper)

def freeze(module):
    for p in module.parameters():
        p.requires_grad = False

# Criterions

In [ ]:
## VarPro for quadratic loss
class ExactRidgeProjection():
    def __init__(self, lmbda, keep_alpha=False):
        self.lmbda = lmbda
        self.keep_alpha = keep_alpha
        if self.keep_alpha:
            self.alpha_list = []
    
    @torch.no_grad()
    def __call__(self, inputs, targets, model, requires_grad=False):
        features = model.feature_model(inputs).clone().detach()
        batch_size, width = features.shape[0], features.shape[1]
        if not self.keep_alpha and batch_size > width: ## underparameterized case
            K = (features.T @ features) / (batch_size * width)
            u = torch.linalg.solve(K + self.lmbda * torch.eye(width),  (features.T @ targets) / batch_size)
        else: ## overparameterized case
            K = (features @ features.T) / (batch_size*width)
            alpha = torch.linalg.solve(K + self.lmbda * torch.eye(batch_size), targets)
            u = features.T @ alpha / batch_size
            if self.keep_alpha:
                self.alpha_list.append(alpha)
        model.outer.weight = nn.Parameter(data=u.view((1,-1)), requires_grad=requires_grad)

class ExactRidgeProjectionUnbiased():
    def __init__(self, lmbda):
        self.lmbda = lmbda
    
    @torch.no_grad()
    def __call__(self, inputs, targets, model, requires_grad=False):
        features = model.feature_model(inputs).clone().detach()
        batch_size, width = features.shape[0], features.shape[1]
        K = (features.T @ features) / (batch_size * width)
        u = torch.linalg.solve(K + self.lmbda * torch.eye(width),  (features.T @ targets) / batch_size + self.lmbda)
        model.outer.weight = nn.Parameter(data=u.view((1,-1)), requires_grad=requires_grad)

class ApproximateRidgeProjection():
    def __init__(self, lmbda, method='cg', tol=1e-7):
        self.lmbda = lmbda
        self.method = method
        self.tol = tol

    def __call__(self, inputs, targets, model):
        features = model.feature_model(inputs).clone().detach()
        width = features.shape[1]
        ridge_objective = lambda u: ((features @ u / width - targets)**2).mean() + self.lmbda * (u**2).mean()
        u = minimize(ridge_objective, model.outer.weight.detach().squeeze(), method=self.method, tol=self.tol).x
        model.outer.weight = nn.Parameter(data=u.view((1,-1)), requires_grad=False)

class LeastSquareCriterion(nn.Module):
    def __init__(self, lmbda, projection=None):
        super().__init__()
        self.lmbda = lmbda
        self.projection = projection
        
    def forward(self, inputs, targets, model):
        if self.projection is not None:
            self.projection(inputs, targets, model)
        predictions = model(inputs)
        return 0.5 * ((predictions - targets)**2).mean() / self.lmbda  + 0.5 * (model.outer.weight**2).mean()

class LeastSquareCriterionUnbiased(nn.Module):
    def __init__(self, lmbda, projection=None):
        super().__init__()
        self.lmbda = lmbda
        self.projection = projection
        
    def forward(self, inputs, targets, model):
        if self.projection is not None:
            self.projection(inputs, targets, model)
        predictions = model(inputs)
        return 0.5 * ((predictions - targets)**2).mean() / self.lmbda  + 0.5 * ((model.outer.weight-1)**2).mean()

class Normalization():
    def __call__(self, w):
        return w / torch.norm(w, 2, dim=-1, keepdim=True).expand_as(w)    

class PeriodicBoundaryCondition():
    def __init__(self, x_min=-1, x_max=1):
        self.x_min = x_min
        self.x_max = x_max

    def __call__(self, w):
        return self.x_min + (w-self.x_min) % (self.x_max-self.x_min)

class BallClipper():
    def __init__(self, radius=1):
        self.radius = radius

    def __call__(self, w):
        norm = w.norm(dim=1, keepdim=True).expand_as(w)
        return (w / norm) * pmin(norm, self.radius)

# thresholding weights value
class Thresholding():
    def __init__(self, x_min=-1, x_max=1):
        self.x_min = x_min
        self.x_max = x_max

    def __call__(self, w):
        return pmax(pmin(w, self.x_max), self.x_min)
        
# Apply clipping function on feature model weights
class FeatureClipper():
    def __init__(self, clipping_function):
        self.clipping_function = clipping_function
        
    @torch.no_grad()
    def __call__(self, model):
        w = model.state_dict()['feature_model.weight']
        w.copy_(self.clipping_function(w))

# Apply clipping function on feature model weights and biases (only for SHL)
class FeatureBiasClipper():
    def __init__(self, weight_clipper, bias_clipper):
        self.weight_clipper = weight_clipper
        self.bias_clipper = bias_clipper
        
    @torch.no_grad()
    def __call__(self, model):
        dico = model.state_dict()
        w, b = dico['feature_model.weight'], dico['feature_model.bias']
        w.copy_(self.weight_clipper(w))
        b.copy_(self.bias_clipper(b))

# Apply clipping function on feature model weights and signs (only for SignedSHL)
class FeatureSignClipper():
    def __init__(self, weight_clipper, sign_clipper):
        self.weight_clipper = weight_clipper
        self.sign_clipper = sign_clipper
        
    @torch.no_grad()
    def __call__(self, model):
        dico = model.state_dict()
        w, s = dico['feature_model.weight'], dico['feature_model.sign']
        w.copy_(self.weight_clipper(w))
        s.copy_(self.sign_clipper(s))

# Distances

In [ ]:
def relu_kernel_matrix(w1, w2):
    Theta = torch.acos(pmax(pmin(w1 @ w2.T, 1), -1))
    return (torch.sin(Theta) + (np.pi-Theta) * torch.cos(Theta)) / (2*np.pi)

class GaussianKernel():
    def __init__(self, gamma=1):
        self.gamma = gamma
        
    def __call__(self, x):
        return torch.exp(-x.norm(dim=-1)**2 / self.gamma)

class EnergyKernel():
    def __call__(self, x):
        return -x.norm(dim=-1)

class DistanceMMD():
    def __init__(self, kernel=EnergyKernel(), projection=nn.Identity()):
        self.kernel = kernel
        self.projection = projection

    def __call__(self, m1, m2, c1, c2):
        K1 = self.kernel(self.projection(m1[:,None,:] - m1[None,:,:]))
        K2 = self.kernel(self.projection(m2[:,None,:] - m2[None,:,:]))
        K3 = self.kernel(self.projection(m1[:,None,:] - m2[None,:,:]))
        return (c1.dot(K1@c1) + c2.dot(K2@c2) - 2*c1.dot(K3@c2)).sqrt()

class DistanceOT():
    def __init__(self, projection=nn.Identity()):
        self.projection = projection

    def __call__(self, m1, m2, c1, c2):
        M = (self.projection(m1[:,None,:] - m2[None,:,:])**2).sum(dim=-1).numpy()
        return np.sqrt(ot.emd2(c1.numpy(), c2.numpy(), M))

def compute_distance(distance, weight_list, weight_ref, c_list=None, c_ref=None, N_eval=100):

    if c_list is None:
        c = torch.ones(weight_list[0].shape[0]) / weight_list[0].shape[0]
        c_list = [c for _ in range(len(weight_list))]
    if c_ref is None:
        c_ref = torch.ones(weight_ref.shape[0]) / weight_ref.shape[0]

    distance_list = []
    idx = np.array([int(i) for i in np.linspace(0, len(weight_list)-1, N_eval+1)])
    for i in tqdm(idx):
        distance_list.append(distance(weight_ref, weight_list[i], c_ref, c_list[i]).item())
    return distance_list, idx

# Other

In [ ]:
class TeacherCosFunction():
    def __call__(self, inputs):
        norm = inputs.norm(dim=1)
        cos = inputs[:,0] / norm
        return 0.5 * norm * cos

class TeacherGammaFunction():
    def __init__(self, gamma):
        self.gamma = gamma

    def __call__(self, inputs):
        rotated_inputs = inputs @ torch.tensor([[1, 0], [0, (1+self.gamma)**0.5]])
        norm = rotated_inputs.norm(dim=1)
        cos = rotated_inputs[:,0] / norm
        sin = rotated_inputs[:,1] / norm
        r1 = cos * torch.asinh(cos*gamma**0.5)
        r2 = sin * torch.asin(sin*(gamma/(1+gamma))**0.5)
        return norm * (r1+r2) / (np.pi*gamma**0.5)

class TeacherStepFunction():
    def __init__(self, alpha, eps):
        self.alpha = alpha
        self.eps = eps
        self.beta = (1-2*(np.pi-eps)*alpha) / (2*eps)
        #assert self.beta > 0

    def __call__(self, inputs):
        inputs_array = inputs.clone().detach().numpy()
        norm = np.linalg.norm(inputs_array, axis=1)
        cos = inputs_array[:,0] / norm
        sin = inputs_array[:,1] / norm
        theta = 2 * np.arctan(sin / (1+cos))
        
        res = ((theta > -0.5*np.pi-self.eps)*(theta < -0.5*np.pi+self.eps)) * (1+np.sin(theta+self.eps))
        res += ((theta > 0.5*np.pi-self.eps)*(theta < 0.5*np.pi+self.eps)) * (1-np.sin(theta-self.eps))
        res += ((theta > -0.5*np.pi+self.eps)*(theta < 0.5*np.pi-self.eps)) * (np.sin(theta+self.eps)-np.sin(theta-self.eps))
    
        return torch.tensor((2 * self.alpha + (self.beta-self.alpha) * res) * norm, dtype=torch.float32)

## Data
d = 1
N = 2048
gamma = 10

alpha = 0.2 * (1 / 6.28)
eps = 0.1

teacher_function = TeacherGammaFunction(gamma)

inputs = torch.randn(N,2)
#rotated_inputs = inputs @ torch.tensor([[0, -1], [1, 0]], dtype=torch.float32)
#targets = 0.5*(teacher_function(inputs) + teacher_function(rotated_inputs)).view((N, 1))
targets = teacher_function(inputs).view((N, 1))
dataset = CustomDataset(inputs, targets)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset))

In [ ]:
class TeacherStudentProblem(LearningProblem):
    def __init__(self, student, teacher, inputs, optimizer, criterion):
        self.teacher = teacher
        self.dataset = CustomDataset(inputs, self.teacher(inputs).clone().detach())
        train_loader = torch.utils.data.DataLoader(self.dataset, batch_size=len(self.dataset))
        super().__init__(student, train_loader, optimizer, criterion)
        
        self.distance_dict = {}
        
    def compute_distance(self, distance, weight_ref=None, c_ref=None, name='teacher', distance_name=None, N_eval=100):
        
        weight_list = [dico['feature_model.0.weight'] for dico in self.state_list]
        
        if weight_ref is None:
            name = 'teacher'
            weight_ref = self.teacher.feature_model[0].weight.clone().detach()
            c_ref = self.teacher.outer.weight.detach().squeeze()
            c_ref.div_(c_ref.norm(p=1))
            
        distance_list, idx = compute_distance(distance, weight_list, weight_ref, c_ref=c_ref, N_eval=N_eval)
        step = len(self.loss_list) // (len(self.state_list)-1)

        self.distance_dict[name] = {'distance_name': distance_name, 'distance_list': distance_list, 'idx': step*idx}